In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import sys

In [ ]:
import os

In [ ]:
#basedir = "/home/marco/git/webvalley/datapreproc"
basedir = "C:/Users/julix/webvalley_git/full_project/data/PLIC-Milano"

# Import the smallest dataset

The plic-milano-foglio-piccolo.xlsx is PLIC_1445_V1_V2_V3_V4_linear variables _ updated 15_01_19.xlsx
and the plic-milano-foglio-grande.xlsx is PLIC_dataset parametri clinici_V1_V2_V3_V4_1445.xlsx

In [ ]:
smalldf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-piccolo.xlsx"), index_col=0).fillna(-1)

In [ ]:
smalldf.head()

In [ ]:
smalldf.info()

# What is male and what is female?
### Should at least be written in the docs of the whole software otherwise it will be impossible to import new data

In [ ]:
smalldf.sesso.unique()

# Import the biggest dataset

In [ ]:
bigdf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-grande.xlsx"), index_col=0).fillna(-1)

In [ ]:
bigdf.info()

In [ ]:
bigdf.head()

# At least one column is not in the biggest dataset

In [ ]:
smalldfcols = set(smalldf.columns.values)

In [ ]:
bigdfcols = set(bigdf.columns.values)

In [ ]:
f1_cols_in_f2 = smalldfcols.intersection(bigdfcols)

In [ ]:
f1_cols_non_in_f2 = smalldfcols.difference(bigdfcols)

In [ ]:
f1_cols_in_f2

In [ ]:
f1_cols_non_in_f2

In [ ]:
len(f1_cols_in_f2)

In [ ]:
len(f1_cols_non_in_f2)

Checking if column names differ because of case sensitivity

In [ ]:
bigdfcols_lower = set([k.lower() for k in bigdfcols])

In [ ]:
smalldfcols_lower = set([k.lower() for k in smalldfcols])

In [ ]:
len(smalldfcols_lower.intersection(bigdfcols_lower))

which is not the case

# Check how many patients in the smaller dataset are also in the biggest one

In [ ]:
paz_in_smalldf = set(smalldf.index.values)

In [ ]:
paz_in_bigdf = set(bigdf.index.values)

In [ ]:
paz_in_smalldf.difference(paz_in_bigdf)

In [ ]:
paz_in_bigdf.difference(paz_in_smalldf)

*All the patients in the small dataset are also in the biggest one*

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()

# Fix known issues with the dataset

The two datasets uses different ways to store the data (M/F -> 0/1; Yes/No -> 1/0)

In [ ]:
smalldf["sesso"] = smalldf["sesso"].map({1: "F", 0: "M"})

*We dropped all sex errors so the method is appropriate; now replace even Si and No*

In [ ]:
# Check all the columns that contains at least one Si or No

bool_cols = list()

for col in f1_cols_in_f2:
    k = getattr(bigdf, col).values
    if np.isin("Sì", k) or np.isin("No", k):
        bool_cols.append(col)

In [ ]:
bool_cols

In [ ]:
# Replace em all

for col in bool_cols:
    smalldf[col] = smalldf[col].map({0: "No", 1: "Sì"})

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()


## Now check if dataset 1 and dataset 2 shared patients contains same data

In [ ]:
# Pick every patient in the big dataset and check if the values are the same
# that are stored in the smaller one

for paz_to_be_checked in paz_in_smalldf:
    paz_from_big = bigdf.loc[paz_to_be_checked]
    paz_from_small = smalldf.loc[paz_to_be_checked]
    if not np.array_equal(paz_from_big[f1_cols_in_f2].values, paz_from_small[f1_cols_in_f2].values):
        print("Paz %s has differences" % paz_to_be_checked)
print("check complete")

# Adding non-duplicate data from smaller dataframe to bigger dataframe

## Preparing smaller dataframe to join with bigger dataframe

In [ ]:
smalldf.info()

In [ ]:
smalldf.drop(f1_cols_in_f2, axis=1, inplace=True)

In [ ]:
smalldf.info()

In [ ]:
bigdf.info()

In [ ]:
joined_df = bigdf.join(smalldf, how='outer')

In [ ]:
joined_df.info()

# Translate variable names

In [ ]:
[x for x in joined_df.columns]

In [ ]:

print("The end.")